In [ ]:
# Necessary Imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
import pickle
from torch.optim.lr_scheduler import StepLR

# import data
data = pickle.load(open("data.p", "rb"))
labels = pickle.load(open("labels.p", "rb"))

In [3]:
# normalize the data by feature

for i in range(len(data[0])):
    # Calculate the mean and std for each attribute
    mean = np.mean(data[0:, i])
    std = np.std(data[0:, i])
    # subtract the mean and divide by the std for each attribute
    for j in range(len(data[0:, i])):
        data[j][i] -= mean
        data[j][i] /= std
        

In [4]:
# split data into train and test
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.1, random_state=10)
train_data = torch.from_numpy(train_data)
test_data = torch.from_numpy(test_data)
train_labels = torch.from_numpy(train_labels)
test_labels = torch.from_numpy(test_labels)

In [23]:
# Define the network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(64, 128)
        self.dropout1 = nn.Dropout(0.8)
        self.fc2 = nn.Linear(128, 16)
        self.dropout2 = nn.Dropout(0.8)
        self.fc3 = nn.Linear(16, 1)


    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
#        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
#        x = self.dropout2(x)
        output = self.fc3(x)
        return output

In [110]:
epochs = 5
batch_size = 1
g = 0.7
learn = 1.0

def train(model, data, labels, device, optimizer, epoch, log_int):
    model.train()
    for i in range(0, int(len(data)/batch_size)):
        inp = data[i*batch_size:(i+1)*batch_size].to(device).float()
        target = labels[i*batch_size:(i+1)*batch_size].to(device).float()
        optimizer.zero_grad()
        out = model(inp)
        loss = nn.L1Loss()(out, target.view(-1,1))
        loss.backward()
        optimizer.step()
        if i % log_int == 0:
            print('Train Epoch: ' + str(epoch) + ' | Loss: ' + str(loss.item()))
def test(model, data, labels, device):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i in range(0, int(len(data))):
            inp = data[i].to(device).float()
            target = labels[i].to(device).float()
            output = model(inp)
            test_loss += nn.L1Loss()(output, target.view(1))
    print("Test loss: " + str(test_loss / len(data)))
        # For each entry in the test data set
            # Run the entry through and add the loss to our total loss
        # Divide by the total length to get the average error

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=learn)
scheduler = StepLR(optimizer, step_size=1, gamma=g)

In [113]:
for i in range(1):
    train(model, train_data, train_labels, device, optimizer, i, 5000)
    test(model, test_data, test_labels, device)
    scheduler.step()

Train Epoch: 0 | Loss: 0.007740763947367668
Train Epoch: 0 | Loss: 7.387585639953613
Train Epoch: 0 | Loss: 0.056192196905612946
Train Epoch: 0 | Loss: 1.3819118738174438
Train Epoch: 0 | Loss: 0.021603059023618698
Train Epoch: 0 | Loss: 0.022214312106370926
Train Epoch: 0 | Loss: 0.024014201015233994
Train Epoch: 0 | Loss: 0.025617457926273346
Train Epoch: 0 | Loss: 0.027203582227230072
Train Epoch: 0 | Loss: 0.028695933520793915
Train Epoch: 0 | Loss: 0.03005155175924301
Train Epoch: 0 | Loss: 0.03138356655836105
Train Epoch: 0 | Loss: 3.522111654281616
Train Epoch: 0 | Loss: 0.049866706132888794
Train Epoch: 0 | Loss: 0.03516776114702225
Train Epoch: 0 | Loss: 0.03639453649520874
Train Epoch: 0 | Loss: 0.03753840923309326
Train Epoch: 0 | Loss: 0.06353600323200226
Train Epoch: 0 | Loss: 0.07384303957223892
Test loss: tensor(0.2844, device='cuda:0')


In [114]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
reg = LinearRegression().fit(train_data, train_labels)
reg.score(train_data, train_labels)

0.05606008329882872

In [115]:
pred = reg.predict(test_data)

In [ ]:
out = model(test_data.to(device).float())

In [120]:
mean_absolute_error(pred, test_labels)

0.39854032006137796

In [119]:
# We see that our model is not perfect, but it does outperforms the basic linear regression
test(model, test_data, test_labels, device)

Test loss: tensor(0.2844, device='cuda:0')
